# Extracting molecular properties with Kudi

This tutorial demonstrates how to explore Gaussian intrinsic reaction coordinate (IRC) data using the public `kudi` API.


## Setup

* Python 3.11
* Install project dependencies with `pip install -e .` from the repository root before running this notebook.

* Run the notebook from inside the `tutorials` directory (or the notebook's own folder) so Python picks up the installed `kudi` package from `src/`.


In [ ]:

from kudi import IRCPath


from pathlib import Path


def find_fixture(filename: str = "gaussian_irc.log") -> Path:
    """Locate a tutorial fixture regardless of the working directory."""
    search_roots = [Path.cwd(), *Path.cwd().parents]
    for root in search_roots:
        for candidate in (root / "tutorials" / "fixtures" / filename, root / "fixtures" / filename):
            if candidate.exists():
                return candidate
    raise FileNotFoundError(
        f"Could not find {filename}; check that the tutorials/fixtures folder is available."
    )

fixture_path = find_fixture()
print(f"Using fixture: {fixture_path}")


## Load the example IRC output

The example log file shipped in `tutorials/fixtures/gaussian_irc.log` contains a two-point IRC path. We parse it with `IRCPath.from_file`, which handles Gaussian output directly.


In [ ]:

irc_path = IRCPath.from_file(fixture_path)

print(f"Parsed {len(irc_path.points)} IRC points")
print("Reaction coordinates:", irc_path.rx_coords)


## Energies along the reaction coordinate

The helper method `relative_energies_kcal` converts Hartree energies into kcal/mol relative to the lowest-energy point. The bar chart below is text-based to avoid extra plotting dependencies.


In [ ]:

relative_energies = irc_path.relative_energies_kcal()
scale = 0
if relative_energies:
    max_energy = max(relative_energies)
    scale = 40 / max_energy if max_energy else 1

for coord, energy in zip(irc_path.rx_coords, relative_energies):
    bar = "█" * int(energy * scale)
    print(f"{coord:6.2f} -> {energy:8.3f} kcal/mol | {bar}")


## Orbital energies

Gaussian logs often include occupied and virtual orbital energies for each IRC point. The `IRCPath` object exposes these as lists of floats. Here we inspect the first point and extract the HOMO/LUMO values.


In [ ]:

first_point = irc_path.points[0]
occ = first_point.occ_orbitals or []
virt = first_point.virt_orbitals or []

print(f"Occupied orbitals ({len(occ)} values):", occ)
print(f"Virtual orbitals ({len(virt)} values):", virt)

if occ and virt:
    homo = occ[-1]
    lumo = virt[0]
    print(f"HOMO: {homo:.3f} Hartree | LUMO: {lumo:.3f} Hartree | Gap: {lumo - homo:.3f} Hartree")
